In [3]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
# Definir la URL de la página web a scrapear
url = "https://www.fctables.com/todays-match-predictions/"

# Obtener el contenido de la página web
response = requests.get(url)

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Buscar los elementos de la página web que queremos extraer
elementos = soup.find_all(class_="table table-striped table-bordered stage-table table-condensed")

df = pd.DataFrame({"Matches":[],
                    "Over HT 1.5 local":[],
                    "Over HT 1.5 visitante":[],
                    "Over HT 1.5 total":[]})

if elementos[0]:
  tbody = elementos[0].find("tbody")
  trs =tbody.find_all("tr")
  for tr in trs:
    url_match = tr.find_all("td")[2].find("a")["href"]
    matchh = tr.find_all("td")[2].find("a").text
    #print(url_match)
    response = requests.get("https://www.fctables.com"+url_match)
    soup = BeautifulSoup(response.content, "html.parser")
    tablas = soup.find_all(id="team_stats_vs")

    for tabla in tablas:
      tbody = tabla.find("tbody")
      trs =tbody.find_all("tr")
      for tr in trs:
        #print(tr)
        if "Over HT 1.5" in tr.text:
          tds = tr.find_all("td")
          if tds[0].text:
            local = tds[0].text
            valor = local[local.find("(") + 1:local.find(")")]
            valor = valor.replace("%","")
            local = int(valor)
          else:
            local = 0
          if tds[2].text:
            visitante = tds[2].text
            valor = visitante[visitante.find("(") + 1:visitante.find(")")]
            valor = valor.replace("%","")
            visitante = int(valor)
          else:
            visitante = 0
          if local > 50 and visitante > 50 :
            df = df.append( pd.DataFrame({"Matches":[matchh],
                    "Over HT 1.5 local":[local],
                    "Over HT 1.5 visitante":[visitante],
                    "Over HT 1.5 total":[ (local*visitante) / 2]}) )
            dic = {'encuentro':matchh,'local':local,'visitante':visitante , 'total': local+visitante }

display(df)
current_date = dt.date.today()
today = current_date.strftime("%w_%B_%Y")
df.to_excel("over15_"+today+".xlsx")


,Matches,Over HT 1.5 local,Over HT 1.5 visitante,Over HT 1.5 total
